In [10]:
import numpy as np
import pandas as pd

# For reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# Date range for synthetic data
START_DATE = "2024-01-01"
END_DATE = "2024-12-31"

# Departments in scope
DEPARTMENTS = ["ER", "Resp_OPD"]

# Shifts used for staffing
SHIFTS = ["morning", "evening", "night"]

# ============================================================================
# STEP 1: Generate External Factors
# ============================================================================

def add_epidemic_wave(df, start_date, length=10):
    """Adds a wave like: 0,1,2,3,2,1,... over 'length' days."""
    pattern = [0, 1, 2, 3, 2, 1]
    idxs = df.index[df["date"].between(start_date, start_date + pd.Timedelta(days=length - 1))]
    for i, idx in enumerate(idxs):
        df.at[idx, "epidemic_alert_level"] = max(
            df.at[idx, "epidemic_alert_level"],
            pattern[i % len(pattern)]
        )

def generate_external_factors(start_date=START_DATE, end_date=END_DATE, seed=42):
    np.random.seed(seed)
    
    dates = pd.date_range(start_date, end_date, freq="D")
    df = pd.DataFrame({"date": dates})
    df["day_of_week"] = df["date"].dt.day_name()
    df["month"] = df["date"].dt.month
    
    # Festivals / holidays
    festival_info = {
        "2024-01-26": "Republic_Day",
        "2024-03-25": "Holi",
        "2024-08-15": "Independence_Day",
        "2024-11-01": "Diwali",
        "2024-12-25": "Christmas",
    }
    festival_dates = pd.to_datetime(list(festival_info.keys()))
    df["holiday_flag"] = df["date"].isin(festival_dates).astype(int)
    df["festival_name"] = df["date"].map(
        lambda d: festival_info.get(d.strftime("%Y-%m-%d"), "")
    )
    
    # AQI
    month_base_aqi = {
        1: 260, 2: 220, 3: 200, 4: 180, 5: 170, 6: 150,
        7: 140, 8: 145, 9: 160, 10: 200, 11: 260, 12: 270,
    }
    df["AQI_base"] = df["month"].map(month_base_aqi)
    noise_aqi = np.random.normal(0, 25, size=len(df))
    df["AQI"] = (df["AQI_base"] + noise_aqi).clip(50, 500)
    
    # Extra pollution around Diwali
    diwali_mask = df["festival_name"] == "Diwali"
    for offset in [0, 1, 2]:
        df.loc[diwali_mask.shift(offset, fill_value=False), "AQI"] += 80
    df["AQI"] = df["AQI"].clip(50, 500).round().astype(int)
    
    # Temperature
    month_base_temp = {
        1: 15, 2: 18, 3: 24, 4: 30, 5: 34, 6: 32,
        7: 30, 8: 30, 9: 29, 10: 26, 11: 20, 12: 16,
    }
    df["temp_base"] = df["month"].map(month_base_temp)
    noise_temp = np.random.normal(0, 2, size=len(df))
    df["temp"] = (df["temp_base"] + noise_temp).round(1)
    
    # Rainfall
    df["rainfall"] = 0.0
    monsoon_mask = df["month"].isin([6, 7, 8, 9])
    df.loc[monsoon_mask, "rainfall"] = np.random.gamma(
        shape=2.5, scale=6, size=monsoon_mask.sum()
    ).round(1)
    
    non_monsoon_mask = ~monsoon_mask
    non_monsoon_idx = df.index[non_monsoon_mask]
    rain_chance = np.random.rand(non_monsoon_idx.size)
    light_rain_idx = non_monsoon_idx[rain_chance < 0.3]
    df.loc[light_rain_idx, "rainfall"] = np.random.uniform(
        0.5, 5.0, light_rain_idx.size
    ).round(1)
    
    # Epidemic alert level
    df["epidemic_alert_level"] = 0
    add_epidemic_wave(df, pd.to_datetime("2024-01-05"), length=10)
    add_epidemic_wave(df, pd.to_datetime("2024-07-10"), length=14)
    add_epidemic_wave(df, pd.to_datetime("2024-12-10"), length=12)
    rand_start = np.random.choice(df["date"].iloc[30:-30])
    add_epidemic_wave(df, rand_start, length=10)
    
    df = df[[
        "date", "day_of_week", "holiday_flag", "festival_name",
        "AQI", "temp", "rainfall", "epidemic_alert_level"
    ]]
    
    return df

print("Generating external factors...")
df_ext = generate_external_factors()
df_ext.to_csv("external_factors.csv", index=False)
print(f"✓ Created external_factors.csv with {len(df_ext)} rows")

# ============================================================================
# STEP 2: Generate Patient Visits
# ============================================================================

print("\nGenerating patient visits...")
rows = []

for _, day_row in df_ext.iterrows():
    date = day_row["date"]
    dow = day_row["day_of_week"]
    holiday = day_row["holiday_flag"]
    aqi = day_row["AQI"]
    epidemic = day_row["epidemic_alert_level"]
    
    for hour in range(24):
        for dept in DEPARTMENTS:
            # Base demand
            base = 6 if dept == "ER" else 4
            
            # Time-of-day effects
            if 9 <= hour <= 12:
                base += 4
            if 18 <= hour <= 21:
                base += 3
            if 0 <= hour <= 5:
                base -= 2
            
            # Weekend effect
            if dow in ["Saturday", "Sunday"]:
                base += 2 if dept == "ER" else -1
            
            # Holiday effect
            if holiday:
                base += 5 if dept == "ER" else 3
            
            # Pollution effect
            if dept == "Resp_OPD":
                if aqi > 200:
                    base += 3
                if aqi > 300:
                    base += 5
            
            # Epidemic effect
            if epidemic >= 1:
                base += 2
            if epidemic >= 2:
                base += 4
            if epidemic == 3:
                base += 6
            
            # Add noise
            mean = max(0, base)
            count = int(np.random.normal(loc=mean, scale=2))
            count = max(count, 0)
            
            rows.append([date, hour, dept, count])

df_visits = pd.DataFrame(
    rows, columns=["date", "hour", "department", "patient_count"]
)
df_visits.to_csv("patient_visits.csv", index=False)
print(f"✓ Created patient_visits.csv with {len(df_visits)} rows")

# ============================================================================
# STEP 3: Train-Test Split with Feature Engineering
# ============================================================================

print("\n" + "="*70)
print("PREPARING DATA FOR MODELING")
print("="*70)

# Load data - IMPORTANT: keep_default_na=False to preserve empty strings
df_visits = pd.read_csv("patient_visits.csv", parse_dates=["date"])
df_ext = pd.read_csv("external_factors.csv", parse_dates=["date"], keep_default_na=False)

print(f"\npatient_visits shape: {df_visits.shape}")
print(f"external_factors shape: {df_ext.shape}")



# Aggregate to daily
df_daily = (
    df_visits.groupby("date")["patient_count"]
    .sum()
    .reset_index()
    .rename(columns={"patient_count": "patients"})
)

print(f"\nAfter aggregating patient_visits to daily: {len(df_daily)} rows")

# Merge with external factors
df = df_daily.merge(df_ext, on="date", how="left")
df = df.sort_values("date").reset_index(drop=True)

print(f"After merge: {len(df)} rows")

# Check for any NaN values before feature engineering
print("\nChecking for NaN values BEFORE feature engineering:")
print(df.isnull().sum())

# Feature engineering
df["month"] = df["date"].dt.month
df["is_weekend"] = df["day_of_week"].isin(["Saturday", "Sunday"]).astype(int)
df["festival_flag"] = ((df["holiday_flag"] == 1) | (df["festival_name"] != "")).astype(int)
df["high_AQI_flag"] = (df["AQI"] >= 250).astype(int)

# Lag features
df["patients_lag1"] = df["patients"].shift(1)
df["patients_lag2"] = df["patients"].shift(2)
df["patients_lag7"] = df["patients"].shift(7)

# Rolling mean
df["patients_roll7"] = (
    df["patients"].shift(1).rolling(window=7, min_periods=3).mean()
)

# Check for NaN values AFTER creating lag features
print("\nChecking for NaN values AFTER creating lag features:")
print(df.isnull().sum())

# Show which rows have NaN
print("\nRows with ANY NaN values:")
nan_mask = df.isnull().any(axis=1)
print(f"Number of rows with NaN: {nan_mask.sum()}")
if nan_mask.sum() > 0 and nan_mask.sum() < 20:
    print(df[nan_mask][["date", "patients", "patients_lag1", "patients_lag7", "patients_roll7"]])

# Drop NaN rows
df_model = df.dropna().reset_index(drop=True)

print(f"\nAfter dropna: {len(df_model)} days (dropped {len(df) - len(df_model)} rows)")

if len(df_model) < 100:
    print("\n⚠️ ERROR: Too many rows dropped! Investigating...")
    print("\nFirst 10 rows of merged data:")
    print(df.head(10)[["date", "patients", "day_of_week", "AQI", "temp"]])
    raise ValueError(f"Only {len(df_model)} rows remain after preprocessing. Something is wrong with the data!")

# Prepare features and target
feature_cols = [
    "holiday_flag", "festival_flag", "AQI", "high_AQI_flag",
    "temp", "rainfall", "epidemic_alert_level", "month", "is_weekend",
    "patients_lag1", "patients_lag2", "patients_lag7", "patients_roll7"
]

X = df_model[feature_cols].values
y = df_model["patients"].values

# Train-test split
test_size = 60
train_size = len(df_model) - test_size

X_train = X[:train_size]
X_test = X[train_size:]
y_train = y[:train_size]
y_test = y[train_size:]

train_dates = df_model["date"].iloc[:train_size]
test_dates = df_model["date"].iloc[train_size:]

print("\n" + "-"*70)
print("TRAIN-TEST SPLIT SUMMARY")
print("-"*70)
print(f"Train size: {len(X_train)} samples")
print(f"Test size:  {len(X_test)} samples")
print(f"\nTrain range: {train_dates.iloc[0].strftime('%Y-%m-%d')} to {train_dates.iloc[-1].strftime('%Y-%m-%d')}")
print(f"Test range:  {test_dates.iloc[0].strftime('%Y-%m-%d')} to {test_dates.iloc[-1].strftime('%Y-%m-%d')}")

print("\n" + "-"*70)
print("TRAINING DATA STATISTICS")
print("-"*70)
print(f"Mean daily patients: {y_train.mean():.1f}")
print(f"Std daily patients:  {y_train.std():.1f}")
print(f"Min daily patients:  {y_train.min()}")
print(f"Max daily patients:  {y_train.max()}")

print("\n" + "-"*70)
print("SAMPLE DATA (First 5 days)")
print("-"*70)
print(df_model[["date", "patients", "AQI", "epidemic_alert_level", "patients_lag7"]].head())

print("\n✓ Data preparation complete! Ready for modeling.")

Generating external factors...
✓ Created external_factors.csv with 366 rows

Generating patient visits...
✓ Created patient_visits.csv with 17568 rows

PREPARING DATA FOR MODELING

patient_visits shape: (17568, 4)
external_factors shape: (366, 8)

After aggregating patient_visits to daily: 366 rows
After merge: 366 rows

Checking for NaN values BEFORE feature engineering:
date                    0
patients                0
day_of_week             0
holiday_flag            0
festival_name           0
AQI                     0
temp                    0
rainfall                0
epidemic_alert_level    0
dtype: int64

Checking for NaN values AFTER creating lag features:
date                    0
patients                0
day_of_week             0
holiday_flag            0
festival_name           0
AQI                     0
temp                    0
rainfall                0
epidemic_alert_level    0
month                   0
is_weekend              0
festival_flag           0
high_AQI_fla

In [11]:
!pip install ollama requests
print("✓ Installation complete!")

✓ Installation complete!


You should consider upgrading via the 'C:\Users\Shalini\Projects\Health_AI_Agent\healthcare_agent_env\Scripts\python.exe -m pip install --upgrade pip' command.


In [14]:
import ollama

try:
    resp = ollama.list()
    print("✓ Ollama is running!\n")

    # Case 1: new-style ListResponse object (what you have)
    if hasattr(resp, "models"):
        model_list = resp.models

        print("Available models:")
        for m in model_list:
            # m is a Model object; name is in m.model
            print(f"  - {m.model}")

        model_names = [m.model for m in model_list]

    # Case 2: dict with "models" (older style)
    elif isinstance(resp, dict) and "models" in resp:
        model_list = resp["models"]

        print("Available models:")
        for m in model_list:
            print(f"  - {m.get('name')}")

        model_names = [m.get("name") for m in model_list]

    # Case 3: plain list of models
    elif isinstance(resp, list):
        model_list = resp

        print("Available models:")
        for m in model_list:
            name = getattr(m, "model", getattr(m, "name", str(m)))
            print(f"  - {name}")

        model_names = [
            getattr(m, "model", getattr(m, "name", str(m)))
            for m in model_list
        ]

    else:
        raise ValueError(f"Unexpected Ollama list() format: {type(resp)} → {resp}")

    # Check for llama3 variants
    print("\nChecking for Llama 3 models...")
    llama_models = [n for n in model_names if "llama3" in n.lower()]

    if llama_models:
        print("✓ Found Llama 3 models:")
        for n in llama_models:
            print(f"  - {n}")
    else:
        print("⚠️ No Llama 3 models found. Pull one with e.g.:")
        print("   ollama pull llama3")

except Exception as e:
    print(f"❌ Error: {e}")
    print("\nCheck that Ollama is running. Use:\n  ollama serve")

✓ Ollama is running!

Available models:
  - llama3:latest

Checking for Llama 3 models...
✓ Found Llama 3 models:
  - llama3:latest


In [16]:
# CELL 3: Test basic Ollama query

import ollama

# Simple test query
def test_ollama():
    response = ollama.chat(
        model='llama3:latest',
        messages=[
            {
                'role': 'user',
                'content': 'What is 2+2? Answer in one sentence.'
            }
        ]
    )
    return response['message']['content']

try:
    result = test_ollama()
    print("✓ Ollama is working!")
    print(f"\nResponse: {result}")
except Exception as e:
    print(f"❌ Error: {e}")

✓ Ollama is working!

Response: The answer to the question of what is 2+2 is simply 4.


In [18]:
# CELL 4: Load all healthcare data for the agent

import pandas as pd
import numpy as np

# 1) Load the base CSVs you actually created
df_visits = pd.read_csv("patient_visits.csv", parse_dates=["date"])
df_ext = pd.read_csv("external_factors.csv", parse_dates=["date"], keep_default_na=False)

print("patient_visits rows:", len(df_visits))
print("external_factors rows:", len(df_ext))

# 2) Create daily summary (total patients per day)
df_daily_summary = (
    df_visits.groupby("date")["patient_count"]
    .sum()
    .reset_index()
    .rename(columns={"patient_count": "total_patients"})
)

print("daily_summary rows:", len(df_daily_summary))

# 3) Merge with external factors
df_complete = df_daily_summary.merge(df_ext, on="date", how="left")

# 4) Add derived fields useful for the agent
df_complete["month_name"] = df_complete["date"].dt.strftime("%B")
df_complete["is_weekend"] = df_complete["day_of_week"].isin(["Saturday", "Sunday"])

print(f"\n✓ Loaded {len(df_complete)} days of healthcare data")
print(f"Date range: {df_complete['date'].min().strftime('%Y-%m-%d')} to {df_complete['date'].max().strftime('%Y-%m-%d')}")
print(f"\nColumns available: {df_complete.columns.tolist()}")
print(f"\nSample data:")
print(df_complete.head(3))

patient_visits rows: 17568
external_factors rows: 366
daily_summary rows: 366

✓ Loaded 366 days of healthcare data
Date range: 2024-01-01 to 2024-12-31

Columns available: ['date', 'total_patients', 'day_of_week', 'holiday_flag', 'festival_name', 'AQI', 'temp', 'rainfall', 'epidemic_alert_level', 'month_name', 'is_weekend']

Sample data:
        date  total_patients day_of_week  holiday_flag festival_name  AQI  \
0 2024-01-01             326      Monday             0                272   
1 2024-01-02             322     Tuesday             0                257   
2 2024-01-03             327   Wednesday             0                276   

   temp  rainfall  epidemic_alert_level month_name  is_weekend  
0  15.4       0.0                     0    January       False  
1  15.0       3.1                     0    January       False  
2  15.2       0.0                     0    January       False  


In [19]:
# CELL 5: Create tools for the agent to analyze data

def get_data_summary():
    """Get overall statistics about the healthcare data"""
    summary = {
        "total_days": len(df_complete),
        "avg_daily_patients": df_complete["total_patients"].mean(),
        "max_daily_patients": df_complete["total_patients"].max(),
        "min_daily_patients": df_complete["total_patients"].min(),
        "avg_aqi": df_complete["AQI"].mean(),
        "avg_temp": df_complete["temp"].mean(),
        "total_holidays": df_complete["holiday_flag"].sum(),
        "total_epidemic_days": (df_complete["epidemic_alert_level"] > 0).sum()
    }
    return summary

def get_date_range_data(start_date, end_date):
    """Get data for a specific date range"""
    mask = (df_complete["date"] >= start_date) & (df_complete["date"] <= end_date)
    return df_complete[mask]

def get_high_patient_days(threshold=500):
    """Get days with patient count above threshold"""
    return df_complete[df_complete["total_patients"] >= threshold]

def get_correlation_insights():
    """Get correlations between patients and environmental factors"""
    correlations = {
        "AQI_correlation": df_complete[["total_patients", "AQI"]].corr().iloc[0, 1],
        "temp_correlation": df_complete[["total_patients", "temp"]].corr().iloc[0, 1],
        "epidemic_correlation": df_complete[["total_patients", "epidemic_alert_level"]].corr().iloc[0, 1]
    }
    return correlations

def get_monthly_stats():
    """Get statistics grouped by month"""
    monthly = df_complete.groupby("month_name").agg({
        "total_patients": ["mean", "max", "min"],
        "AQI": "mean",
        "temp": "mean"
    }).round(2)
    return monthly

# Test the tools
print("Testing data analysis tools...\n")

print("1. Data Summary:")
print(get_data_summary())

print("\n2. Correlation Insights:")
print(get_correlation_insights())

print("\n3. High Patient Days (>500):")
high_days = get_high_patient_days(500)
print(f"Found {len(high_days)} days with >500 patients")

print("\n✓ All tools are working!")

Testing data analysis tools...

1. Data Summary:
{'total_days': 366, 'avg_daily_patients': np.float64(319.8169398907104), 'max_daily_patients': np.int64(919), 'min_daily_patients': np.int64(212), 'avg_aqi': np.float64(197.06830601092895), 'avg_temp': np.float64(25.268852459016397), 'total_holidays': np.int64(5), 'total_epidemic_days': np.int64(35)}

2. Correlation Insights:
{'AQI_correlation': np.float64(0.34588933713998143), 'temp_correlation': np.float64(-0.3487974665165451), 'epidemic_correlation': np.float64(0.8809398204665864)}

3. High Patient Days (>500):
Found 26 days with >500 patients

✓ All tools are working!


In [20]:
# CELL 6: Create the Healthcare AI Agent

import ollama
import json

class HealthcareAgent:
    def __init__(self, model='llama3:latest'):
        self.model = model
        self.conversation_history = []
        
        # System prompt with context about available data
        self.system_prompt = """You are a helpful healthcare data analyst assistant. 
You have access to hospital patient visit data for the year 2024, including:
- Daily patient counts (ER and Respiratory OPD)
- Environmental factors (AQI, temperature, rainfall)
- Epidemic alert levels (0-3)
- Holidays and festivals
- Day of week information

When users ask questions, analyze the data context provided and give clear, 
concise insights. Focus on actionable information for hospital management.
Keep responses brief and to the point (2-3 sentences max unless asked for detail)."""
        
    def create_data_context(self, user_query):
        """Create relevant data context based on the user's query"""
        context_parts = []
        
        # Always include basic summary
        summary = get_data_summary()
        context_parts.append(f"Dataset: {summary['total_days']} days in 2024")
        context_parts.append(f"Avg daily patients: {summary['avg_daily_patients']:.0f}")
        context_parts.append(f"Range: {summary['min_daily_patients']:.0f} to {summary['max_daily_patients']:.0f} patients/day")
        
        # Add correlations if query mentions AQI, pollution, temperature, etc.
        keywords_env = ['aqi', 'pollution', 'air quality', 'temperature', 'weather', 'epidemic']
        if any(kw in user_query.lower() for kw in keywords_env):
            corr = get_correlation_insights()
            context_parts.append(f"\nCorrelations with patient count:")
            context_parts.append(f"- AQI: {corr['AQI_correlation']:.3f}")
            context_parts.append(f"- Temperature: {corr['temp_correlation']:.3f}")
            context_parts.append(f"- Epidemic level: {corr['epidemic_correlation']:.3f}")
        
        # Add high patient day info if relevant
        keywords_high = ['high', 'peak', 'busy', 'most', 'surge', 'spike']
        if any(kw in user_query.lower() for kw in keywords_high):
            high_days = get_high_patient_days(500)
            if len(high_days) > 0:
                context_parts.append(f"\nHigh-volume days (>500 patients): {len(high_days)}")
                top_day = high_days.nlargest(1, 'total_patients').iloc[0]
                context_parts.append(f"Highest day: {top_day['date'].strftime('%Y-%m-%d')} with {top_day['total_patients']:.0f} patients")
        
        # Add monthly trends if relevant
        keywords_monthly = ['month', 'seasonal', 'trend', 'pattern']
        if any(kw in user_query.lower() for kw in keywords_monthly):
            monthly = get_monthly_stats()
            context_parts.append(f"\nMonthly averages available for analysis")
        
        return "\n".join(context_parts)
    
    def query(self, user_message):
        """Send a query to the agent"""
        # Create data context
        data_context = self.create_data_context(user_message)
        
        # Build messages
        messages = [
            {'role': 'system', 'content': self.system_prompt},
            {'role': 'system', 'content': f"Data Context:\n{data_context}"}
        ]
        
        # Add conversation history
        messages.extend(self.conversation_history)
        
        # Add current user message
        messages.append({'role': 'user', 'content': user_message})
        
        # Get response from Ollama
        response = ollama.chat(
            model=self.model,
            messages=messages
        )
        
        assistant_message = response['message']['content']
        
        # Update conversation history
        self.conversation_history.append({'role': 'user', 'content': user_message})
        self.conversation_history.append({'role': 'assistant', 'content': assistant_message})
        
        return assistant_message
    
    def reset(self):
        """Clear conversation history"""
        self.conversation_history = []
        print("✓ Conversation history cleared")

# Initialize the agent
agent = HealthcareAgent()

print("✓ Healthcare AI Agent initialized!")
print("\nThe agent is ready to answer questions about your hospital data.")
print("\nExample questions you can ask:")
print("  - What were the busiest days?")
print("  - How does AQI affect patient visits?")
print("  - What's the trend during epidemic alerts?")
print("  - Which months had the highest patient load?")

✓ Healthcare AI Agent initialized!

The agent is ready to answer questions about your hospital data.

Example questions you can ask:
  - What were the busiest days?
  - How does AQI affect patient visits?
  - What's the trend during epidemic alerts?
  - Which months had the highest patient load?


In [21]:
# CELL 7: Test the agent with sample queries

def test_agent():
    """Run some test queries"""
    
    test_questions = [
        "What's the average daily patient count in our hospital?",
        "How does air quality (AQI) impact patient visits?",
        "What were the busiest days in 2024?"
    ]
    
    print("="*70)
    print("TESTING HEALTHCARE AI AGENT")
    print("="*70)
    
    for i, question in enumerate(test_questions, 1):
        print(f"\n[Q{i}] {question}")
        print("-"*70)
        
        try:
            response = agent.query(question)
            print(f"[Agent] {response}")
        except Exception as e:
            print(f"❌ Error: {e}")
        
        print()
    
    print("="*70)
    print("✓ Testing complete!")

# Run the tests
test_agent()

TESTING HEALTHCARE AI AGENT

[Q1] What's the average daily patient count in our hospital?
----------------------------------------------------------------------
[Agent] Based on the data, the average daily patient count in your hospital is approximately 320 patients.


[Q2] How does air quality (AQI) impact patient visits?
----------------------------------------------------------------------
[Agent] The AQI has a moderate positive correlation with patient visits, with a coefficient of 0.346. This suggests that as AQI levels increase, so do patient visits to your hospital.


[Q3] What were the busiest days in 2024?
----------------------------------------------------------------------
[Agent] According to the data, the busiest days in 2024 had daily patient counts ranging from 919 to 850 patients. These peak volumes likely require additional staffing and resources for efficient management of the increased demand.

✓ Testing complete!


In [22]:
# CELL 8: Interactive chat interface

def chat_with_agent():
    """Interactive chat loop"""
    print("="*70)
    print("HEALTHCARE AI AGENT - INTERACTIVE MODE")
    print("="*70)
    print("\nType your questions and press Enter.")
    print("Commands: 'reset' to clear history, 'quit' to exit\n")
    
    while True:
        user_input = input("You: ").strip()
        
        if not user_input:
            continue
            
        if user_input.lower() in ['quit', 'exit', 'q']:
            print("\n👋 Goodbye!")
            break
            
        if user_input.lower() == 'reset':
            agent.reset()
            continue
        
        try:
            response = agent.query(user_input)
            print(f"\nAgent: {response}\n")
        except Exception as e:
            print(f"\n❌ Error: {e}\n")

# Uncomment the line below to start interactive chat
chat_with_agent()

print("✓ Interactive chat function ready!")
print("\nTo start chatting, run: chat_with_agent()")

HEALTHCARE AI AGENT - INTERACTIVE MODE

Type your questions and press Enter.
Commands: 'reset' to clear history, 'quit' to exit



You:  quit



👋 Goodbye!
✓ Interactive chat function ready!

To start chatting, run: chat_with_agent()


In [32]:
# CELL 9: Enhanced agent that can execute data queries directly

class AdvancedHealthcareAgent(HealthcareAgent):
    """Enhanced agent that can call tools to fetch actual data"""
    
    def __init__(self, model='llama3:latest'):
        super().__init__(model)    
        
        # Update system prompt to include tool capabilities
        self.system_prompt = """You are a helpful healthcare data analyst assistant with access to real hospital data.

When answering questions, you can use these tools:
1. get_data_summary() - Get overall statistics
2. get_high_patient_days(threshold) - Find busy days
3. get_correlation_insights() - Get correlations with environmental factors
4. get_monthly_stats() - Get monthly breakdown

Provide clear, data-driven insights. Be concise but informative."""
    
    def query_with_tools(self, user_message):
        """Query that can execute tools and use real results"""
        
        # First, determine what tools to use based on the query
        tools_to_execute = []
        results = {}
        
        # Simple keyword matching to decide which tools to use
        query_lower = user_message.lower()
        
        if any(kw in query_lower for kw in ['average', 'total', 'overall', 'summary', 'statistics']):
            tools_to_execute.append('summary')
            results['summary'] = get_data_summary()
        
        if any(kw in query_lower for kw in ['busy', 'peak', 'high', 'most patients']):
            tools_to_execute.append('high_days')
            results['high_days'] = get_high_patient_days(500).to_dict('records')[:5]  # Top 5
        
        if any(kw in query_lower for kw in ['correlation', 'relationship', 'affect', 'impact', 'aqi', 'pollution', 'temperature']):
            tools_to_execute.append('correlations')
            results['correlations'] = get_correlation_insights()
        
        if any(kw in query_lower for kw in ['month', 'monthly', 'seasonal']):
            tools_to_execute.append('monthly')
            monthly_df = get_monthly_stats()
            results['monthly'] = monthly_df.to_dict()
        
        # Build context with actual data results
        context_parts = ["Tool Results:"]
        
        if 'summary' in results:
            s = results['summary']
            context_parts.append(f"\nOverall Statistics:")
            context_parts.append(f"- Total days: {s['total_days']}")
            context_parts.append(f"- Avg patients/day: {s['avg_daily_patients']:.1f}")
            context_parts.append(f"- Max: {s['max_daily_patients']}, Min: {s['min_daily_patients']}")
            context_parts.append(f"- Avg AQI: {s['avg_aqi']:.1f}")
            context_parts.append(f"- Days with epidemic alerts: {s['total_epidemic_days']}")
        
        if 'correlations' in results:
            c = results['correlations']
            context_parts.append(f"\nCorrelations with patient count:")
            context_parts.append(f"- AQI: {c['AQI_correlation']:.3f}")
            context_parts.append(f"- Temperature: {c['temp_correlation']:.3f}")
            context_parts.append(f"- Epidemic level: {c['epidemic_correlation']:.3f}")
        
        if 'high_days' in results and results['high_days']:
            context_parts.append(f"\nTop busy days:")
            for day in results['high_days'][:3]:
                date_str = pd.to_datetime(day['date']).strftime('%Y-%m-%d')
                context_parts.append(f"- {date_str}: {day['total_patients']:.0f} patients (AQI: {day['AQI']}, Epidemic: {day['epidemic_alert_level']})")
        
        data_context = "\n".join(context_parts)
        
        # Build messages for Ollama
        messages = [
            {'role': 'system', 'content': self.system_prompt},
            {'role': 'system', 'content': data_context}
        ]
        
        messages.extend(self.conversation_history)
        messages.append({'role': 'user', 'content': user_message})
        
        # Get response
        response = ollama.chat(model=self.model, messages=messages)
        assistant_message = response['message']['content']
        
        # Update history
        self.conversation_history.append({'role': 'user', 'content': user_message})
        self.conversation_history.append({'role': 'assistant', 'content': assistant_message})
        
        return assistant_message

# Create advanced agent
advanced_agent = AdvancedHealthcareAgent()

print("✓ Advanced Healthcare Agent initialized!")
print("\nThis agent can execute tools and provide data-driven answers.")
print("\nTry asking:")
print("  - What's the correlation between AQI and patient visits?")
print("  - Show me the busiest days and explain why")
print("  - What are the monthly trends?")

✓ Advanced Healthcare Agent initialized!

This agent can execute tools and provide data-driven answers.

Try asking:
  - What's the correlation between AQI and patient visits?
  - Show me the busiest days and explain why
  - What are the monthly trends?


In [33]:
# CELL 10: Test the advanced agent with complex queries

def test_advanced_agent():
    """Test advanced agent with data-driven queries"""
    
    print("="*70)
    print("TESTING ADVANCED HEALTHCARE AI AGENT")
    print("="*70)
    
    questions = [
        "What's the relationship between air quality and patient visits?",
        "Show me the 3 busiest days and explain what might have caused the surge",
        "How do patient numbers vary by month throughout the year?"
    ]
    
    for i, question in enumerate(questions, 1):
        print(f"\n{'='*70}")
        print(f"[Question {i}] {question}")
        print('='*70)
        
        try:
            response = advanced_agent.query_with_tools(question)
            print(f"\n[Agent Response]\n{response}\n")
        except Exception as e:
            print(f"❌ Error: {e}")
            import traceback
            traceback.print_exc()
    
    print("="*70)
    print("✓ Advanced agent testing complete!")

# Run the test
test_advanced_agent()

TESTING ADVANCED HEALTHCARE AI AGENT

[Question 1] What's the relationship between air quality and patient visits?

[Agent Response]
Based on our data, it appears that there is a moderate positive correlation (AQI: 0.346) between Air Quality Index (AQI) and patient visits. This suggests that as AQI levels increase (i.e., air quality worsens), the number of patient visits tends to increase as well.

While this correlation does not imply causation, it may be worth exploring potential factors contributing to this relationship. For example, poor air quality can exacerbate respiratory issues, potentially leading to increased hospitalizations and patient visits.


[Question 2] Show me the 3 busiest days and explain what might have caused the surge

[Agent Response]
Using our data analysis tool `get_high_patient_days(threshold)`, I've identified the top 3 busiest days with a threshold of 150 patients or more:

1. January 10th: 165 patients
2. February 22nd: 158 patients
3. March 15th: 155 pat

In [34]:
# CELL 11: Build a predictive model for patient forecasting

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

print("="*70)
print("BUILDING PREDICTIVE MODEL")
print("="*70)

# We already have X_train, X_test, y_train, y_test from earlier
# Let's train a Random Forest model

model = RandomForestRegressor(
    n_estimators=100,
    max_depth=10,
    random_state=42,
    n_jobs=-1
)

print("\nTraining Random Forest model...")
model.fit(X_train, y_train)

# Evaluate on test set
y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"\n✓ Model trained successfully!")
print(f"\nTest Set Performance:")
print(f"  MAE:  {mae:.2f} patients")
print(f"  RMSE: {rmse:.2f} patients")

# Feature importance
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

print(f"\nTop 5 Most Important Features:")
print(feature_importance.head())

# Function to predict future patient load
def predict_patient_load(date, external_data, historical_data):
    """
    Predict patient load for a given date
    
    Args:
        date: datetime for prediction
        external_data: dict with AQI, temp, rainfall, epidemic_alert_level
        historical_data: recent patient counts for lag features
    """
    # Extract features
    features = []
    
    # Holiday and festival flags (simplified - you'd check actual calendar)
    features.append(0)  # holiday_flag
    features.append(0)  # festival_flag
    
    # External factors
    features.append(external_data['AQI'])
    features.append(1 if external_data['AQI'] >= 250 else 0)  # high_AQI_flag
    features.append(external_data['temp'])
    features.append(external_data.get('rainfall', 0))
    features.append(external_data.get('epidemic_alert_level', 0))
    
    # Calendar features
    features.append(date.month)
    features.append(1 if date.weekday() >= 5 else 0)  # is_weekend
    
    # Lag features
    features.append(historical_data.get('lag1', 0))
    features.append(historical_data.get('lag2', 0))
    features.append(historical_data.get('lag7', 0))
    features.append(historical_data.get('roll7', 0))
    
    # Predict
    X_pred = np.array(features).reshape(1, -1)
    prediction = model.predict(X_pred)[0]
    
    return prediction

print("\n✓ Prediction function ready!")

BUILDING PREDICTIVE MODEL

Training Random Forest model...

✓ Model trained successfully!

Test Set Performance:
  MAE:  25.81 patients
  RMSE: 36.08 patients

Top 5 Most Important Features:
                feature  importance
6  epidemic_alert_level    0.793701
2                   AQI    0.100424
1         festival_flag    0.037609
0          holiday_flag    0.032964
8            is_weekend    0.009840

✓ Prediction function ready!


In [35]:
# CELL 12: Define autonomous decision-making rules

class AutonomousDecisionEngine:
    """Makes decisions based on predictions and thresholds"""
    
    def __init__(self):
        # Thresholds for different alert levels
        self.thresholds = {
            'low': 300,
            'normal': 400,
            'high': 500,
            'critical': 600
        }
        
        # Staffing ratios (patients per staff member)
        self.staffing_ratios = {
            'doctors': 50,      # 1 doctor per 50 patients
            'nurses': 20,       # 1 nurse per 20 patients
            'support': 40       # 1 support staff per 40 patients
        }
        
        # Supply usage rates (per patient per day)
        self.supply_rates = {
            'oxygen_cylinder': 0.08,
            'nebulizer_session': 0.05,
            'inhaler': 0.25,
            'paracetamol_strip': 0.7,
            'PPE_kit': 0.5,
            'masks': 1.5,
            'gloves': 3.0
        }
        
        # Safety stock days
        self.safety_stock_days = 7
    
    def classify_load(self, predicted_patients):
        """Classify predicted patient load"""
        if predicted_patients < self.thresholds['low']:
            return 'LOW'
        elif predicted_patients < self.thresholds['normal']:
            return 'NORMAL'
        elif predicted_patients < self.thresholds['high']:
            return 'HIGH'
        else:
            return 'CRITICAL'
    
    def recommend_staffing(self, predicted_patients):
        """Calculate recommended staffing levels"""
        recommendations = {}
        
        # Calculate required staff
        doctors_needed = max(4, int(np.ceil(predicted_patients / self.staffing_ratios['doctors'])))
        nurses_needed = max(8, int(np.ceil(predicted_patients / self.staffing_ratios['nurses'])))
        support_needed = max(4, int(np.ceil(predicted_patients / self.staffing_ratios['support'])))
        
        # Distribute across shifts (rough distribution)
        recommendations['morning'] = {
            'doctors': int(doctors_needed * 0.4),
            'nurses': int(nurses_needed * 0.4),
            'support': int(support_needed * 0.4)
        }
        recommendations['evening'] = {
            'doctors': int(doctors_needed * 0.35),
            'nurses': int(nurses_needed * 0.35),
            'support': int(support_needed * 0.35)
        }
        recommendations['night'] = {
            'doctors': doctors_needed - recommendations['morning']['doctors'] - recommendations['evening']['doctors'],
            'nurses': nurses_needed - recommendations['morning']['nurses'] - recommendations['evening']['nurses'],
            'support': support_needed - recommendations['morning']['support'] - recommendations['evening']['support']
        }
        
        return recommendations
    
    def recommend_supplies(self, predicted_patients, current_stock):
        """Calculate supply requirements and reorder recommendations"""
        recommendations = {}
        
        for item, usage_rate in self.supply_rates.items():
            # Expected usage
            daily_usage = predicted_patients * usage_rate
            weekly_usage = daily_usage * self.safety_stock_days
            
            # Check current stock
            current = current_stock.get(item, 0)
            
            # Should we reorder?
            if current < weekly_usage:
                reorder_qty = int(np.ceil(weekly_usage * 2 - current))  # Reorder to 2 weeks supply
                recommendations[item] = {
                    'current_stock': current,
                    'daily_usage': daily_usage,
                    'reorder': True,
                    'reorder_quantity': reorder_qty,
                    'urgency': 'HIGH' if current < daily_usage * 2 else 'MEDIUM'
                }
            else:
                recommendations[item] = {
                    'current_stock': current,
                    'daily_usage': daily_usage,
                    'reorder': False,
                    'days_remaining': int(current / daily_usage) if daily_usage > 0 else 999
                }
        
        return recommendations
    
    def generate_patient_advisory(self, predicted_patients, external_factors):
        """Generate patient advisory messages"""
        advisories = []
        
        load_level = self.classify_load(predicted_patients)
        
        # High load advisory
        if load_level in ['HIGH', 'CRITICAL']:
            advisories.append({
                'type': 'CAPACITY_WARNING',
                'severity': 'HIGH' if load_level == 'CRITICAL' else 'MEDIUM',
                'message': f"Expected high patient volume ({int(predicted_patients)} patients). Non-emergency cases may experience longer wait times.",
                'recommendation': "Defer non-urgent visits if possible. Call ahead to schedule appointments."
            })
        
        # AQI advisory
        if external_factors.get('AQI', 0) > 250:
            advisories.append({
                'type': 'AIR_QUALITY',
                'severity': 'HIGH',
                'message': f"Poor air quality (AQI: {external_factors['AQI']}). Increased respiratory cases expected.",
                'recommendation': "Avoid outdoor activities. Wear N95 masks. Vulnerable populations should stay indoors."
            })
        
        # Epidemic advisory
        if external_factors.get('epidemic_alert_level', 0) >= 2:
            advisories.append({
                'type': 'EPIDEMIC_ALERT',
                'severity': 'CRITICAL',
                'message': f"Epidemic alert level {external_factors['epidemic_alert_level']}. Follow safety protocols.",
                'recommendation': "Maintain social distancing. Wear masks. Get vaccinated. Seek medical attention if symptomatic."
            })
        
        return advisories

# Initialize decision engine
decision_engine = AutonomousDecisionEngine()

print("✓ Autonomous Decision Engine initialized!")
print("\nDecision thresholds configured:")
print(f"  Low load: < {decision_engine.thresholds['low']} patients")
print(f"  Normal load: {decision_engine.thresholds['low']}-{decision_engine.thresholds['normal']} patients")
print(f"  High load: {decision_engine.thresholds['normal']}-{decision_engine.thresholds['high']} patients")
print(f"  Critical load: > {decision_engine.thresholds['high']} patients")

✓ Autonomous Decision Engine initialized!

Decision thresholds configured:
  Low load: < 300 patients
  Normal load: 300-400 patients
  High load: 400-500 patients
  Critical load: > 500 patients


In [36]:
# CELL 13: Autonomous AI Agent that combines ML predictions with LLM reasoning

class AutonomousHealthcareAgent:
    """
    Fully autonomous agent that:
    1. Makes predictions using ML model
    2. Applies decision rules
    3. Uses LLM for contextual reasoning
    4. Generates and executes actions
    """
    
    def __init__(self, model, decision_engine, llm_model='llama3:latest'):
        self.predictor = model
        self.decision_engine = decision_engine
        self.llm_model = llm_model
        self.action_log = []
        
    def analyze_and_act(self, date, external_data, historical_data, current_stock, auto_execute=False):
        """
        Main autonomous loop:
        1. Predict patient load
        2. Make decisions
        3. Use LLM for reasoning
        4. Generate actions
        5. (Optionally) Execute actions
        """
        
        print("\n" + "="*70)
        print(f"AUTONOMOUS ANALYSIS FOR {date.strftime('%Y-%m-%d')}")
        print("="*70)
        
        # Step 1: Predict patient load
        predicted_load = predict_patient_load(date, external_data, historical_data)
        load_level = self.decision_engine.classify_load(predicted_load)
        
        print(f"\n[PREDICTION]")
        print(f"  Expected patients: {int(predicted_load)}")
        print(f"  Load level: {load_level}")
        
        # Step 2: Generate recommendations
        staffing_rec = self.decision_engine.recommend_staffing(predicted_load)
        supply_rec = self.decision_engine.recommend_supplies(predicted_load, current_stock)
        advisories = self.decision_engine.generate_patient_advisory(predicted_load, external_data)
        
        print(f"\n[STAFFING RECOMMENDATIONS]")
        for shift, staff in staffing_rec.items():
            print(f"  {shift.capitalize()}: {staff['doctors']}D / {staff['nurses']}N / {staff['support']}S")
        
        print(f"\n[SUPPLY RECOMMENDATIONS]")
        reorder_items = {k: v for k, v in supply_rec.items() if v['reorder']}
        if reorder_items:
            for item, info in reorder_items.items():
                print(f"  {item}: REORDER {info['reorder_quantity']} units (urgency: {info['urgency']})")
        else:
            print(f"  All supplies adequate")
        
        print(f"\n[PATIENT ADVISORIES]")
        if advisories:
            for adv in advisories:
                print(f"  [{adv['severity']}] {adv['type']}: {adv['message']}")
        else:
            print(f"  No special advisories")
        
        # Step 3: Use LLM for contextual reasoning and action planning
        llm_context = self._build_llm_context(
            date, predicted_load, load_level, external_data, 
            staffing_rec, supply_rec, advisories
        )
        
        llm_response = self._query_llm(llm_context)
        
        print(f"\n[LLM REASONING]")
        print(f"{llm_response}")
        
        # Step 4: Create action plan
        actions = self._create_action_plan(
            date, predicted_load, load_level, staffing_rec, 
            supply_rec, advisories, llm_response
        )
        
        print(f"\n[ACTION PLAN]")
        for i, action in enumerate(actions, 1):
            print(f"  {i}. {action['action_type']}: {action['description']}")
        
        # Step 5: Execute (if auto_execute=True)
        if auto_execute:
            print(f"\n[EXECUTING ACTIONS]")
            for action in actions:
                self._execute_action(action)
                print(f"  ✓ Executed: {action['action_type']}")
        else:
            print(f"\n[ACTIONS READY] Set auto_execute=True to execute automatically")
        
        # Log this analysis
        self.action_log.append({
            'date': date,
            'predicted_load': predicted_load,
            'load_level': load_level,
            'actions': actions,
            'executed': auto_execute
        })
        
        return {
            'prediction': predicted_load,
            'load_level': load_level,
            'staffing': staffing_rec,
            'supplies': supply_rec,
            'advisories': advisories,
            'actions': actions
        }
    
    def _build_llm_context(self, date, predicted_load, load_level, external_data, 
                           staffing_rec, supply_rec, advisories):
        """Build context for LLM reasoning"""
        
        context = f"""You are an autonomous healthcare operations AI analyzing data for {date.strftime('%Y-%m-%d')}.

PREDICTIONS:
- Expected patient load: {int(predicted_load)} patients
- Load classification: {load_level}

ENVIRONMENTAL FACTORS:
- AQI: {external_data.get('AQI', 'N/A')}
- Temperature: {external_data.get('temp', 'N/A')}°C
- Epidemic alert level: {external_data.get('epidemic_alert_level', 0)}

STAFFING RECOMMENDATIONS:
{self._format_staffing(staffing_rec)}

SUPPLY STATUS:
{self._format_supplies(supply_rec)}

PATIENT ADVISORIES:
{self._format_advisories(advisories)}

Based on this analysis, provide:
1. A brief risk assessment (2-3 sentences)
2. Key priorities for hospital management
3. Any additional proactive measures to consider

Keep your response concise and actionable."""
        
        return context
    
    def _query_llm(self, context):
        """Query LLM for reasoning"""
        try:
            response = ollama.chat(
                model=self.llm_model,
                messages=[
                    {'role': 'system', 'content': 'You are an autonomous healthcare operations AI assistant. Provide concise, actionable insights.'},
                    {'role': 'user', 'content': context}
                ]
            )
            return response['message']['content']
        except Exception as e:
            return f"LLM reasoning unavailable: {e}"
    
    def _format_staffing(self, staffing_rec):
        lines = []
        for shift, staff in staffing_rec.items():
            lines.append(f"  {shift}: {staff['doctors']} doctors, {staff['nurses']} nurses, {staff['support']} support")
        return "\n".join(lines)
    
    def _format_supplies(self, supply_rec):
        lines = []
        reorder_items = {k: v for k, v in supply_rec.items() if v['reorder']}
        if reorder_items:
            for item, info in reorder_items.items():
                lines.append(f"  {item}: LOW STOCK - reorder {info['reorder_quantity']} units")
        else:
            lines.append("  All supplies adequate")
        return "\n".join(lines)
    
    def _format_advisories(self, advisories):
        if not advisories:
            return "  None"
        lines = []
        for adv in advisories:
            lines.append(f"  [{adv['severity']}] {adv['message']}")
        return "\n".join(lines)
    
    def _create_action_plan(self, date, predicted_load, load_level, staffing_rec, 
                            supply_rec, advisories, llm_reasoning):
        """Create executable action plan"""
        actions = []
        
        # Action 1: Update staffing schedule
        actions.append({
            'action_type': 'STAFFING_UPDATE',
            'description': f'Schedule staff for {load_level} load: {sum(s["doctors"] for s in staffing_rec.values())} doctors total',
            'data': staffing_rec,
            'priority': 'HIGH' if load_level in ['HIGH', 'CRITICAL'] else 'NORMAL'
        })
        
        # Action 2: Supply orders
        reorder_items = {k: v for k, v in supply_rec.items() if v['reorder']}
        if reorder_items:
            actions.append({
                'action_type': 'SUPPLY_ORDER',
                'description': f'Order {len(reorder_items)} supply items',
                'data': reorder_items,
                'priority': 'HIGH' if any(v['urgency'] == 'HIGH' for v in reorder_items.values()) else 'NORMAL'
            })
        
        # Action 3: Patient communications
        if advisories:
            actions.append({
                'action_type': 'PATIENT_ADVISORY',
                'description': f'Issue {len(advisories)} advisory messages',
                'data': advisories,
                'priority': 'HIGH' if any(a['severity'] == 'CRITICAL' for a in advisories) else 'NORMAL'
            })
        
        # Action 4: Alert management if critical
        if load_level == 'CRITICAL':
            actions.append({
                'action_type': 'MANAGEMENT_ALERT',
                'description': 'Notify hospital management of critical load prediction',
                'data': {'predicted_load': predicted_load, 'llm_reasoning': llm_reasoning},
                'priority': 'CRITICAL'
            })
        
        return actions
    
    def _execute_action(self, action):
        """Execute an action (placeholder - would interface with real systems)"""
        # In production, this would:
        # - Update databases
        # - Send emails/SMS
        # - Update scheduling systems
        # - Trigger procurement workflows
        # - Post advisories to website/app
        
        # For now, we just log it
        print(f"    [SIMULATED] {action['action_type']}: {action['description']}")
        
        # You would add real execution logic here, e.g.:
        # if action['action_type'] == 'STAFFING_UPDATE':
        #     update_staff_database(action['data'])
        # elif action['action_type'] == 'SUPPLY_ORDER':
        #     submit_procurement_order(action['data'])
        # etc.
    
    def run_continuous_monitoring(self, forecast_days=7):
        """Run autonomous monitoring for next N days"""
        print("\n" + "="*70)
        print(f"STARTING CONTINUOUS AUTONOMOUS MONITORING ({forecast_days} days)")
        print("="*70)
        
        # Get the most recent data
        latest_date = df_model['date'].max()
        
        for day_offset in range(1, forecast_days + 1):
            target_date = latest_date + pd.Timedelta(days=day_offset)
            
            # Get recent historical data for lag features
            recent_data = df_model[df_model['date'] < target_date].tail(7)
            historical_data = {
                'lag1': recent_data.iloc[-1]['patients'] if len(recent_data) >= 1 else 0,
                'lag2': recent_data.iloc[-2]['patients'] if len(recent_data) >= 2 else 0,
                'lag7': recent_data.iloc[-7]['patients'] if len(recent_data) >= 7 else 0,
                'roll7': recent_data['patients'].mean() if len(recent_data) >= 3 else 0
            }
            
            # Simulate external data (in production, get from weather API, etc.)
            external_data = {
                'AQI': 250,  # Example
                'temp': 25,
                'rainfall': 0,
                'epidemic_alert_level': 0
            }
            
            # Simulate current stock
            current_stock = {
                'oxygen_cylinder': 200,
                'nebulizer_session': 300,
                'inhaler': 400,
                'paracetamol_strip': 600,
                'PPE_kit': 400,
                'masks': 800,
                'gloves': 1200
            }
            
            # Run analysis (without auto-execution for demo)
            self.analyze_and_act(target_date, external_data, historical_data, current_stock, auto_execute=False)
            
            print("\n" + "-"*70)

# Initialize autonomous agent
autonomous_agent = AutonomousHealthcareAgent(model, decision_engine)

print("\n✓ Autonomous Healthcare Agent initialized!")
print("\nThis agent can:")
print("  1. Predict patient load using ML")
print("  2. Apply decision rules for staffing/supplies")
print("  3. Use LLM for contextual reasoning")
print("  4. Generate executable action plans")
print("  5. (Optionally) Execute actions automatically")


✓ Autonomous Healthcare Agent initialized!

This agent can:
  1. Predict patient load using ML
  2. Apply decision rules for staffing/supplies
  3. Use LLM for contextual reasoning
  4. Generate executable action plans
  5. (Optionally) Execute actions automatically


In [37]:
# CELL 14: Test the autonomous agent

import pandas as pd

# Test for a single future date
print("="*70)
print("TESTING AUTONOMOUS AGENT - SINGLE DATE ANALYSIS")
print("="*70)

# Pick a date after our training period
test_date = pd.to_datetime('2024-12-01')

# Get historical data for lag features
recent_data = df_model[df_model['date'] < test_date].tail(7)
historical_data = {
    'lag1': recent_data.iloc[-1]['patients'] if len(recent_data) >= 1 else 300,
    'lag2': recent_data.iloc[-2]['patients'] if len(recent_data) >= 2 else 300,
    'lag7': recent_data.iloc[-7]['patients'] if len(recent_data) >= 7 else 300,
    'roll7': recent_data['patients'].mean() if len(recent_data) >= 3 else 300
}

# Simulate high-risk scenario
high_risk_scenario = {
    'AQI': 320,  # Very poor air quality
    'temp': 28,
    'rainfall': 0,
    'epidemic_alert_level': 2  # Active epidemic
}

# Current supply stock (some items low)
current_stock = {
    'oxygen_cylinder': 150,  # Low
    'nebulizer_session': 180,  # Low
    'inhaler': 400,
    'paracetamol_strip': 600,
    'PPE_kit': 250,
    'masks': 800,
    'gloves': 1200
}

# Run autonomous analysis
result = autonomous_agent.analyze_and_act(
    test_date, 
    high_risk_scenario, 
    historical_data, 
    current_stock, 
    auto_execute=False  # Set to True to actually execute actions
)

print("\n" + "="*70)
print("✓ Single date analysis complete!")

TESTING AUTONOMOUS AGENT - SINGLE DATE ANALYSIS

AUTONOMOUS ANALYSIS FOR 2024-12-01

[PREDICTION]
  Expected patients: 589
  Load level: CRITICAL

[STAFFING RECOMMENDATIONS]
  Morning: 4D / 12N / 6S
  Evening: 4D / 10N / 5S
  Night: 4D / 8N / 4S

[SUPPLY RECOMMENDATIONS]
  oxygen_cylinder: REORDER 511 units (urgency: MEDIUM)
  nebulizer_session: REORDER 233 units (urgency: MEDIUM)
  inhaler: REORDER 1665 units (urgency: MEDIUM)
  paracetamol_strip: REORDER 5182 units (urgency: HIGH)
  PPE_kit: REORDER 3880 units (urgency: HIGH)
  masks: REORDER 11589 units (urgency: HIGH)
  gloves: REORDER 23578 units (urgency: HIGH)

[PATIENT ADVISORIES]
  [HIGH] CAPACITY_WARNING: Expected high patient volume (589 patients). Non-emergency cases may experience longer wait times.
  [HIGH] AIR_QUALITY: Poor air quality (AQI: 320). Increased respiratory cases expected.
  [CRITICAL] EPIDEMIC_ALERT: Epidemic alert level 2. Follow safety protocols.

[LLM REASONING]
Risk Assessment:
The hospital is at high ri

In [39]:
# CELL 15: Run continuous autonomous monitoring for 7 days

print("="*70)
print("AUTONOMOUS MONITORING MODE")
print("="*70)
print("\nThe agent will now analyze the next 7 days and make decisions autonomously...")
print("(In production, this would run as a scheduled job every day)")

# Run 7-day forecast
autonomous_agent.run_continuous_monitoring(forecast_days=7)

print("\n" + "="*70)
print("MONITORING SESSION COMPLETE")
print("="*70)

# Show action log summary
print(f"\nTotal analyses performed: {len(autonomous_agent.action_log)}")
print("\nAction summary:")

action_types = {}
for log_entry in autonomous_agent.action_log:
    for action in log_entry['actions']:
        action_type = action['action_type']
        action_types[action_type] = action_types.get(action_type, 0) + 1

for action_type, count in action_types.items():
    print(f"  {action_type}: {count} times")

print("\n✓ Autonomous monitoring complete!")
print("\nTo enable automatic execution, set auto_execute=True in the analyze_and_act() method.")

AUTONOMOUS MONITORING MODE

The agent will now analyze the next 7 days and make decisions autonomously...
(In production, this would run as a scheduled job every day)

STARTING CONTINUOUS AUTONOMOUS MONITORING (7 days)

AUTONOMOUS ANALYSIS FOR 2025-01-01

[PREDICTION]
  Expected patients: 317
  Load level: NORMAL

[STAFFING RECOMMENDATIONS]
  Morning: 2D / 6N / 3S
  Evening: 2D / 5N / 2S
  Night: 3D / 5N / 3S

[SUPPLY RECOMMENDATIONS]
  inhaler: REORDER 712 units (urgency: MEDIUM)
  paracetamol_strip: REORDER 2512 units (urgency: MEDIUM)
  PPE_kit: REORDER 1823 units (urgency: MEDIUM)
  masks: REORDER 5867 units (urgency: HIGH)
  gloves: REORDER 12134 units (urgency: HIGH)

[PATIENT ADVISORIES]
  No special advisories

[LLM REASONING]
Based on the analysis, here are the recommendations:

**Risk Assessment:** The AQI is moderate at 250, which may pose minor respiratory issues for patients with pre-existing conditions. However, with no epidemic alert level raised, the overall risk remain

In [41]:
!pip install schedule

You should consider upgrading via the 'C:\Users\Shalini\Projects\Health_AI_Agent\healthcare_agent_env\Scripts\python.exe -m pip install --upgrade pip' command.


In [42]:
# CELL 16: Production-ready autonomous agent with scheduling

import schedule
import time
from datetime import datetime, timedelta

class ProductionAutonomousAgent:
    """
    Production-ready autonomous agent that runs on a schedule
    Completely removes humans from the loop
    """
    
    def __init__(self, agent, enable_auto_execution=False):
        self.agent = agent
        self.enable_auto_execution = enable_auto_execution
        self.running = False
        
    def daily_analysis_job(self):
        """Job that runs every day"""
        print(f"\n{'='*70}")
        print(f"SCHEDULED AUTONOMOUS ANALYSIS - {datetime.now()}")
        print(f"{'='*70}")
        
        # Analyze next 3 days
        for days_ahead in [1, 2, 3]:
            target_date = datetime.now() + timedelta(days=days_ahead)
            
            # Get latest data (in production, fetch from live database)
            recent_data = df_model.tail(7)
            historical_data = {
                'lag1': recent_data.iloc[-1]['patients'],
                'lag2': recent_data.iloc[-2]['patients'],
                'lag7': recent_data.iloc[-7]['patients'],
                'roll7': recent_data['patients'].mean()
            }
            
            # Get external data (in production, fetch from APIs)
            external_data = self._fetch_external_data(target_date)
            
            # Get current stock (in production, query inventory system)
            current_stock = self._fetch_current_stock()
            
            # Run analysis with auto-execution if enabled
            self.agent.analyze_and_act(
                target_date,
                external_data,
                historical_data,
                current_stock,
                auto_execute=self.enable_auto_execution
            )
        
        print(f"\n{'='*70}")
        print("✓ Daily autonomous analysis complete")
        print(f"{'='*70}\n")
    
    def _fetch_external_data(self, date):
        """Fetch external data from APIs"""
        # In production, this would call:
        # - Weather API for temperature, rainfall
        # - Air quality API for AQI
        # - Health department API for epidemic alerts
        
        # For demo, return simulated data
        return {
            'AQI': 250,
            'temp': 25,
            'rainfall': 0,
            'epidemic_alert_level': 0
        }
    
    def _fetch_current_stock(self):
        """Fetch current stock from inventory system"""
        # In production, query your inventory database
        
        # For demo, return simulated stock
        return {
            'oxygen_cylinder': 200,
            'nebulizer_session': 300,
            'inhaler': 400,
            'paracetamol_strip': 600,
            'PPE_kit': 400,
            'masks': 800,
            'gloves': 1200
        }
    
    def start_scheduler(self, run_time="09:00"):
        """Start the autonomous agent on a schedule"""
        print(f"\n{'='*70}")
        print("AUTONOMOUS AGENT SCHEDULER STARTING")
        print(f"{'='*70}")
        print(f"\nSchedule: Daily at {run_time}")
        print(f"Auto-execution: {'ENABLED' if self.enable_auto_execution else 'DISABLED'}")
        print(f"\nAgent will autonomously:")
        print("  1. Predict patient load for next 3 days")
        print("  2. Make staffing decisions")
        print("  3. Order supplies if needed")
        print("  4. Issue patient advisories")
        if self.enable_auto_execution:
            print("  5. EXECUTE ALL ACTIONS AUTOMATICALLY")
        else:
            print("  5. Generate action plans (execution disabled)")
        
        print(f"\nPress Ctrl+C to stop the scheduler")
        print(f"{'='*70}\n")
        
        # Schedule the job
        schedule.every().day.at(run_time).do(self.daily_analysis_job)
        
        # Also run immediately on start
        print("Running initial analysis...")
        self.daily_analysis_job()
        
        # Keep running
        self.running = True
        try:
            while self.running:
                schedule.run_pending()
                time.sleep(60)  # Check every minute
        except KeyboardInterrupt:
            print("\n\nScheduler stopped by user")
            self.running = False

# Create production agent (with auto-execution DISABLED for safety)
production_agent = ProductionAutonomousAgent(
    autonomous_agent, 
    enable_auto_execution=False  # Set to True to enable full automation
)

print("✓ Production Autonomous Agent ready!")
print("\nTo start autonomous scheduling:")
print("  production_agent.start_scheduler(run_time='09:00')")
print("\nWARNING: Set enable_auto_execution=True only after thorough testing!")
print("         This will allow the agent to execute actions without human approval.")

✓ Production Autonomous Agent ready!

To start autonomous scheduling:
  production_agent.start_scheduler(run_time='09:00')

         This will allow the agent to execute actions without human approval.
